In [2]:
# Install necessary libraries
!pip install nltk pandas scikit-learn

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

# Download necessary NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load the dataset
data = pd.read_csv('labeled_job_descriptions.csv')

# Preprocessing function
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    return ' '.join(tokens)

data['description'] = data['Job description'].apply(preprocess_text)

# Print unique labels to check for discrepancies
print("Unique labels in the dataset before mapping:", data['Label'].unique())

# Update the label mapping to match your dataset
label_mapping = {
    'Response A': 0,
    'Response B': 1,
    'Response C': 2,
    'Response D': 3,
    'Response E': 4,  # If 'Response E' exists
    'Response F': 5   # If 'Response F' exists
}

# Map the labels
data['Label'] = data['Label'].map(label_mapping)

# Check for any missing values in the Label column after mapping
if data['Label'].isnull().any():
    print("There are missing values in the Label column. Please check the data.")
    # Print rows with missing Labels
    print(data[data['Label'].isnull()])
else:
    # Splitting the data
    X_train, X_test, y_train, y_test = train_test_split(data['description'], data['Label'], test_size=0.2, random_state=42)

    # Building pipelines for different classifiers
    pipelines = {
        'Logistic Regression': Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('clf', LogisticRegression(class_weight='balanced'))
        ]),
        'SVM': Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('clf', SVC(class_weight='balanced', probability=True))
        ]),
        'Random Forest': Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('clf', RandomForestClassifier(class_weight='balanced'))
        ])
    }

    # Define parameter grids for each classifier
    param_grids = {
        'Logistic Regression': {
            'tfidf__max_df': [0.8, 0.9, 1.0],
            'tfidf__min_df': [1, 2, 3],
            'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)],
            'clf__C': [0.1, 1, 10],
            'clf__solver': ['liblinear', 'lbfgs']
        },
        'SVM': {
            'tfidf__max_df': [0.8, 0.9, 1.0],
            'tfidf__min_df': [1, 2, 3],
            'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)],
            'clf__C': [0.1, 1, 10],
            'clf__kernel': ['linear', 'rbf']
        },
        'Random Forest': {
            'tfidf__max_df': [0.8, 0.9, 1.0],
            'tfidf__min_df': [1, 2, 3],
            'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)],
            'clf__n_estimators': [50, 100, 200]
        }
    }

    best_models = {}
    best_accuracy = 0
    best_model_name = None

    # Perform GridSearchCV for each model
    for model_name in pipelines:
        print(f"Training {model_name}...")
        grid_search = GridSearchCV(pipelines[model_name], param_grids[model_name], cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
        grid_search.fit(X_train, y_train)
        best_models[model_name] = grid_search.best_estimator_
        accuracy = grid_search.best_score_
        print(f"{model_name} best params: {grid_search.best_params_}")
        print(f"{model_name} best accuracy: {accuracy}")

        # Check if this model is the best so far
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model_name = model_name

    print(f"\nBest model is {best_model_name} with accuracy of {best_accuracy}")

    # Evaluate the best model on the test set
    best_model = best_models[best_model_name]
    y_pred = best_model.predict(X_test)
    print(f'\n{best_model_name} classification report on test set:')
    print(classification_report(y_test, y_pred))

    # Function to predict the response for a new job description using the best model
    def predict_response(job_description):
        processed_description = preprocess_text(job_description)
        prediction = best_model.predict([processed_description])[0]
        reverse_label_mapping = {v: k for k, v in label_mapping.items()}
        return reverse_label_mapping[prediction]

    # Example usage
    new_job_description = "Manage university financial reports and budget forecasting."
    predicted_response = predict_response(new_job_description)
    print(f'Predicted Response: {predicted_response}')

    new_job_description_other = "Assist in organizing office files and managing schedules."
    predicted_response_other = predict_response(new_job_description_other)
    print(f'Predicted Response: {predicted_response_other}')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nosao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nosao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nosao\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Unique labels in the dataset before mapping: ['Response C' 'Response B' 'Response A' 'Response D']
Training Logistic Regression...
Fitting 5 folds for each of 162 candidates, totalling 810 fits


c:\Users\nosao\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Logistic Regression best params: {'clf__C': 10, 'clf__solver': 'lbfgs', 'tfidf__max_df': 0.8, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2)}
Logistic Regression best accuracy: 0.9385507246376813
Training SVM...
Fitting 5 folds for each of 162 candidates, totalling 810 fits


c:\Users\nosao\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


SVM best params: {'clf__C': 10, 'clf__kernel': 'linear', 'tfidf__max_df': 0.8, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 1)}
SVM best accuracy: 0.9211594202898551
Training Random Forest...
Fitting 5 folds for each of 81 candidates, totalling 405 fits


c:\Users\nosao\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Random Forest best params: {'clf__n_estimators': 50, 'tfidf__max_df': 1.0, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 1)}
Random Forest best accuracy: 0.9206763285024154

Best model is Logistic Regression with accuracy of 0.9385507246376813

Logistic Regression classification report on test set:
              precision    recall  f1-score   support

           1       0.71      0.83      0.77        18
           2       0.90      0.87      0.88        30
           3       1.00      0.78      0.88         9

    accuracy                           0.84        57
   macro avg       0.87      0.83      0.84        57
weighted avg       0.86      0.84      0.84        57

Predicted Response: Response C
Predicted Response: Response C


In [17]:
new_job_description = "The Director of PMO & Strategic Change will lead the delivery of the University’s strategic change portfolio and Programme Management Offices. The Director will work in partnership with senior leaders and key stakeholders across the institution to ensure the delivery of the strategic change programme that underpins delivery of the University’s strategic plan."
predicted_response = predict_response(new_job_description)
print(f'Predicted Response: {predicted_response}')

Predicted Response: Response A


In [3]:
# Install necessary libraries
!pip install nltk pandas scikit-learn

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

# Download necessary NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load the dataset
data = pd.read_csv('labeled_job_descriptions.csv')

# Preprocessing function
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    return ' '.join(tokens)

data['description'] = data['Job description'].apply(preprocess_text)

# Print unique labels to check for discrepancies
print("Unique labels in the dataset before mapping:", data['Label'].unique())

# Update the label mapping to match your dataset
label_mapping = {
    'Response A': 0,
    'Response B': 1,
    'Response C': 2,
    'Response D': 3,
    'Response E': 4,  # If 'Response E' exists
    'Response F': 5   # If 'Response F' exists
}

# Map the labels
data['Label'] = data['Label'].map(label_mapping)

# Check for any missing values in the Label column after mapping
if data['Label'].isnull().any():
    print("There are missing values in the Label column. Please check the data.")
    # Print rows with missing Labels
    print(data[data['Label'].isnull()])
else:
    # Splitting the data
    X_train, X_test, y_train, y_test = train_test_split(data['description'], data['Label'], test_size=0.2, random_state=42)

    # Building pipelines for different classifiers
    pipelines = {
        'Logistic Regression': Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('clf', LogisticRegression(class_weight='balanced'))
        ]),
        'SVM': Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('clf', SVC(class_weight='balanced', probability=True))
        ]),
        'Random Forest': Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('clf', RandomForestClassifier(class_weight='balanced'))
        ])
    }

    # Define parameter grids for each classifier
    param_grids = {
        'Logistic Regression': {
            'tfidf__max_df': [0.8, 0.9, 1.0],
            'tfidf__min_df': [1, 2, 3],
            'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)],
            'clf__C': [0.1, 1, 10],
            'clf__solver': ['liblinear', 'lbfgs']
        },
        'SVM': {
            'tfidf__max_df': [0.8, 0.9, 1.0],
            'tfidf__min_df': [1, 2, 3],
            'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)],
            'clf__C': [0.1, 1, 10],
            'clf__kernel': ['linear', 'rbf']
        },
        'Random Forest': {
            'tfidf__max_df': [0.8, 0.9, 1.0],
            'tfidf__min_df': [1, 2, 3],
            'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)],
            'clf__n_estimators': [50, 100, 200]
        }
    }

    best_models = {}
    best_accuracy = 0
    best_model_name = None
    best_model = None

    # Perform GridSearchCV for each model
    for model_name in pipelines:
        print(f"Training {model_name}...")
        grid_search = GridSearchCV(pipelines[model_name], param_grids[model_name], cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
        grid_search.fit(X_train, y_train)
        best_models[model_name] = grid_search.best_estimator_
        accuracy = grid_search.best_score_
        print(f"{model_name} best params: {grid_search.best_params_}")
        print(f"{model_name} best accuracy: {accuracy}")

        # Evaluate each model on the test set
        y_pred = best_models[model_name].predict(X_test)
        print(f'\n{model_name} classification report on test set:')
        print(classification_report(y_test, y_pred))

        # Check if this model is the best so far
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model_name = model_name
            best_model = best_models[model_name]

    print(f"\nBest model is {best_model_name} with accuracy of {best_accuracy}")

    # Function to predict the response for a new job description using the best model
    def predict_response(job_description):
        processed_description = preprocess_text(job_description)
        prediction = best_model.predict([processed_description])[0]
        reverse_label_mapping = {v: k for k, v in label_mapping.items()}
        return reverse_label_mapping[prediction]

    # Example usage
    new_job_description = "Manage university financial reports and budget forecasting."
    predicted_response = predict_response(new_job_description)
    print(f'Predicted Response: {predicted_response}')

    new_job_description_other = "Assist in organizing office files and managing schedules."
    predicted_response_other = predict_response(new_job_description_other)
    print(f'Predicted Response: {predicted_response_other}')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nosao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nosao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nosao\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Unique labels in the dataset before mapping: ['Response C' 'Response B' 'Response A' 'Response D']
Training Logistic Regression...
Fitting 5 folds for each of 162 candidates, totalling 810 fits


c:\Users\nosao\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Logistic Regression best params: {'clf__C': 10, 'clf__solver': 'lbfgs', 'tfidf__max_df': 0.8, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2)}
Logistic Regression best accuracy: 0.9385507246376813

Logistic Regression classification report on test set:
              precision    recall  f1-score   support

           1       0.71      0.83      0.77        18
           2       0.90      0.87      0.88        30
           3       1.00      0.78      0.88         9

    accuracy                           0.84        57
   macro avg       0.87      0.83      0.84        57
weighted avg       0.86      0.84      0.84        57

Training SVM...
Fitting 5 folds for each of 162 candidates, totalling 810 fits


c:\Users\nosao\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


SVM best params: {'clf__C': 10, 'clf__kernel': 'linear', 'tfidf__max_df': 0.8, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 1)}
SVM best accuracy: 0.9211594202898551

SVM classification report on test set:
              precision    recall  f1-score   support

           1       0.73      0.89      0.80        18
           2       0.93      0.87      0.90        30
           3       1.00      0.78      0.88         9

    accuracy                           0.86        57
   macro avg       0.89      0.84      0.86        57
weighted avg       0.88      0.86      0.86        57

Training Random Forest...
Fitting 5 folds for each of 81 candidates, totalling 405 fits


c:\Users\nosao\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Random Forest best params: {'clf__n_estimators': 200, 'tfidf__max_df': 1.0, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2)}
Random Forest best accuracy: 0.9251207729468598

Random Forest classification report on test set:
              precision    recall  f1-score   support

           1       0.81      0.94      0.87        18
           2       0.96      0.87      0.91        30
           3       1.00      1.00      1.00         9

    accuracy                           0.91        57
   macro avg       0.92      0.94      0.93        57
weighted avg       0.92      0.91      0.91        57


Best model is Logistic Regression with accuracy of 0.9385507246376813
Predicted Response: Response C
Predicted Response: Response C


In [6]:
new_job_description = " To continually review and reflect upon the currency and operation of the University’s Academic Regulations, benchmarking against the sector and regulatory requirements as appropriate, and when required to drive forward any changes necessary to the Academic Regulations."
predicted_response = predict_response(new_job_description)
print(f'Predicted Response: {predicted_response}')

new_job_description_other = "To continually review and reflect upon the currency and operation of the University’s Academic Regulations, benchmarking against the sector and regulatory requirements as appropriate, and when required to drive forward any changes necessary to the Academic Regulations."
predicted_response_other = predict_response(new_job_description_other)
print(f'Predicted Response: {predicted_response_other}')

Predicted Response: Response B
Predicted Response: Response B


In [7]:
# Install necessary libraries
!pip install nltk pandas scikit-learn

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

# Download necessary NLTK datasets
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load the dataset
data = pd.read_csv('labeled_job_descriptions.csv')

# Preprocessing function
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    return ' '.join(tokens)

data['description'] = data['Job description'].apply(preprocess_text)

# Print unique labels to check for discrepancies
print("Unique labels in the dataset before mapping:", data['Label'].unique())

# Update the label mapping to match your dataset
label_mapping = {
    'Response A': 0,
    'Response B': 1,
    'Response C': 2,
    'Response D': 3,
    'Response E': 4,  # If 'Response E' exists
    'Response F': 5   # If 'Response F' exists
}

# Map the labels
data['Label'] = data['Label'].map(label_mapping)

# Check for any missing values in the Label column after mapping
if data['Label'].isnull().any():
    print("There are missing values in the Label column. Please check the data.")
    # Print rows with missing Labels
    print(data[data['Label'].isnull()])
else:
    # Splitting the data
    X_train, X_test, y_train, y_test = train_test_split(data['description'], data['Label'], test_size=0.2, random_state=42)

    # Building pipelines for different classifiers
    pipelines = {
        'Logistic Regression': Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('clf', LogisticRegression(class_weight='balanced'))
        ]),
        'SVM': Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('clf', SVC(class_weight='balanced', probability=True))
        ]),
        'Random Forest': Pipeline([
            ('tfidf', TfidfVectorizer()),
            ('clf', RandomForestClassifier(class_weight='balanced'))
        ])
    }

    # Define parameter grids for each classifier
    param_grids = {
        'Logistic Regression': {
            'tfidf__max_df': [0.8, 0.9, 1.0],
            'tfidf__min_df': [1, 2, 3],
            'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)],
            'clf__C': [0.1, 1, 10],
            'clf__solver': ['liblinear', 'lbfgs']
        },
        'SVM': {
            'tfidf__max_df': [0.8, 0.9, 1.0],
            'tfidf__min_df': [1, 2, 3],
            'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)],
            'clf__C': [0.1, 1, 10],
            'clf__kernel': ['linear', 'rbf']
        },
        'Random Forest': {
            'tfidf__max_df': [0.8, 0.9, 1.0],
            'tfidf__min_df': [1, 2, 3],
            'tfidf__ngram_range': [(1, 1), (1, 2), (2, 2)],
            'clf__n_estimators': [50, 100, 200]
        }
    }

    best_models = {}
    best_accuracy = 0
    best_model_name = None
    best_model = None

    # Perform GridSearchCV for each model
    for model_name in pipelines:
        print(f"Training {model_name}...")
        grid_search = GridSearchCV(pipelines[model_name], param_grids[model_name], cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
        grid_search.fit(X_train, y_train)
        best_models[model_name] = grid_search.best_estimator_
        accuracy = grid_search.best_score_
        print(f"{model_name} best params: {grid_search.best_params_}")
        print(f"{model_name} best accuracy: {accuracy}")

        # Evaluate each model on the test set
        y_pred = best_models[model_name].predict(X_test)
        print(f'\n{model_name} classification report on test set:')
        print(classification_report(y_test, y_pred))

        # Check if this model is the best so far
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model_name = model_name
            best_model = best_models[model_name]

    print(f"\nBest model is {best_model_name} with accuracy of {best_accuracy}")

    # Function to predict the response for multiple job descriptions using the best model
    def predict_responses(job_descriptions):
        processed_descriptions = [preprocess_text(desc) for desc in job_descriptions]
        predictions = best_model.predict(processed_descriptions)
        reverse_label_mapping = {v: k for k, v in label_mapping.items()}
        return [reverse_label_mapping[prediction] for prediction in predictions]

    # Example usage
    job_descriptions = [
        "Manage university financial reports and budget forecasting.",
        "Assist in organizing office files and managing schedules."
    ]
    predicted_responses = predict_responses(job_descriptions)
    for job_desc, response in zip(job_descriptions, predicted_responses):
        print(f'Job Description: {job_desc}\nPredicted Response: {response}\n')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nosao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nosao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nosao\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Unique labels in the dataset before mapping: ['Response C' 'Response B' 'Response A' 'Response D']
Training Logistic Regression...
Fitting 5 folds for each of 162 candidates, totalling 810 fits


c:\Users\nosao\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Logistic Regression best params: {'clf__C': 10, 'clf__solver': 'lbfgs', 'tfidf__max_df': 0.8, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2)}
Logistic Regression best accuracy: 0.9385507246376813

Logistic Regression classification report on test set:
              precision    recall  f1-score   support

           1       0.71      0.83      0.77        18
           2       0.90      0.87      0.88        30
           3       1.00      0.78      0.88         9

    accuracy                           0.84        57
   macro avg       0.87      0.83      0.84        57
weighted avg       0.86      0.84      0.84        57

Training SVM...
Fitting 5 folds for each of 162 candidates, totalling 810 fits


c:\Users\nosao\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


SVM best params: {'clf__C': 10, 'clf__kernel': 'linear', 'tfidf__max_df': 0.8, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 1)}
SVM best accuracy: 0.9211594202898551

SVM classification report on test set:
              precision    recall  f1-score   support

           1       0.73      0.89      0.80        18
           2       0.93      0.87      0.90        30
           3       1.00      0.78      0.88         9

    accuracy                           0.86        57
   macro avg       0.89      0.84      0.86        57
weighted avg       0.88      0.86      0.86        57

Training Random Forest...
Fitting 5 folds for each of 81 candidates, totalling 405 fits


c:\Users\nosao\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Random Forest best params: {'clf__n_estimators': 100, 'tfidf__max_df': 0.8, 'tfidf__min_df': 3, 'tfidf__ngram_range': (1, 2)}
Random Forest best accuracy: 0.9207729468599034

Random Forest classification report on test set:
              precision    recall  f1-score   support

           1       0.73      0.89      0.80        18
           2       0.93      0.87      0.90        30
           3       1.00      0.78      0.88         9

    accuracy                           0.86        57
   macro avg       0.89      0.84      0.86        57
weighted avg       0.88      0.86      0.86        57


Best model is Logistic Regression with accuracy of 0.9385507246376813
Job Description: Manage university financial reports and budget forecasting.
Predicted Response: Response C

Job Description: Assist in organizing office files and managing schedules.
Predicted Response: Response C



In [11]:
    # Function to predict the response for multiple job descriptions using the best model
def predict_responses(job_descriptions):
  processed_descriptions = [preprocess_text(desc) for desc in job_descriptions]
  predictions = best_model.predict(processed_descriptions)
  reverse_label_mapping = {v: k for k, v in label_mapping.items()}
  return [reverse_label_mapping[prediction] for prediction in predictions]  

# Example usage
job_descriptions = [
"Manage university financial reports and budget forecasting.",
"Assist in organizing office files and managing schedules.",
"To plan, manage and deliver student recruitment events based on-campus and virtually..",
"	To monitor, review and report on the impact and effectiveness of all student recruitment events…",
"The Director of PMO & Strategic Change will lead the delivery of the University’s strategic change portfolio and Programme Management Offices. The Director will work in partnership with senior leaders and key stakeholders across the institution to ensure the delivery of the strategic change programme that underpins delivery of the University’s strategic plan.",
"To contribute to continuing improvements to management and financial systems and to the maintenance of effective administration."

]
predicted_responses = predict_responses(job_descriptions)
for job_desc, response in zip(job_descriptions, predicted_responses):
  print(f'Job Description: {job_desc}\nPredicted Response: {response}\n')

Job Description: Manage university financial reports and budget forecasting.
Predicted Response: Response C

Job Description: Assist in organizing office files and managing schedules.
Predicted Response: Response C

Job Description: To plan, manage and deliver student recruitment events based on-campus and virtually..
Predicted Response: Response C

Job Description: 	To monitor, review and report on the impact and effectiveness of all student recruitment events…
Predicted Response: Response B

Job Description: The Director of PMO & Strategic Change will lead the delivery of the University’s strategic change portfolio and Programme Management Offices. The Director will work in partnership with senior leaders and key stakeholders across the institution to ensure the delivery of the strategic change programme that underpins delivery of the University’s strategic plan.
Predicted Response: Response A

Job Description: To contribute to continuing improvements to management and financial syst

In [14]:
    # Function to predict the response for multiple job descriptions using the best model
def predict_responses(job_descriptions):
  processed_descriptions = [preprocess_text(desc) for desc in job_descriptions]
  predictions = best_model.predict(processed_descriptions)
  reverse_label_mapping = {v: k for k, v in label_mapping.items()}
  return [reverse_label_mapping[prediction] for prediction in predictions]  

# Example usage
job_descriptions = [
"To provide of an effective Joinery resource to ensure the University fabric is efficiently maintained on a day-to-day basis including undertaking Project works. To ensure the effective interaction of Estate and Facilities services with other services"
]
predicted_responses = predict_responses(job_descriptions)
for job_desc, response in zip(job_descriptions, predicted_responses):
  print(f'Job Description: {job_desc}\nPredicted Response: {response}\n')

Job Description: To provide of an effective Joinery resource to ensure the University fabric is efficiently maintained on a day-to-day basis including undertaking Project works. To ensure the effective interaction of Estate and Facilities services with other services
Predicted Response: Response B

